In [9]:
from openai import OpenAI
client = OpenAI(
    api_key = "",
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)


response = client.chat.completions.create(
    model="gemini-1.5-flash",
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Explain to me how AI works"
        }
    ]
)

print(response.choices[0].message)

ChatCompletionMessage(content='Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, it aims to create systems that can perform tasks that typically require human intelligence.  These tasks include things like learning, reasoning, problem-solving, perception, and natural language understanding.  There\'s no single "how" AI works, as different AI approaches use different methods.  However, we can break down some common underlying principles:\n\n**1. Data is King:**  Most AI systems rely heavily on data.  The more relevant and high-quality data you feed an AI system, the better it will perform.  This data is used to train the AI model.\n\n**2. Algorithms are the Engine:**  Algorithms are sets of rules and statistical techniques that the AI uses to process the data. These algorithms allow the AI to learn patterns, make predictions, and improve its performance over time.  Different algorithms are suited to different tasks.  Examples include:\n\n* **Ma

In [2]:
from pydantic import BaseModel
from typing import Optional, Literal

class ParameterProperty(BaseModel):
    type: str
    description: str
    enum: Optional[list[str]] = None


class Parameters(BaseModel):
    type: Literal["object"]
    properties: dict[str, ParameterProperty]
    required: Optional[list[str]]


class Function(BaseModel):
    name: str
    description: str
    parameters: Parameters


class Tool(BaseModel):
    type: Literal["function"]
    function: Function

In [3]:
tools = [Tool(
    type="function",
    function=Function(
        name="search_weaviate",
        description="Search a Database containing information about Weaviate.",
        parameters=Parameters(
            type="object",
            properties={
                "search_query": ParameterProperty(
                    type="string",
                    description="The natural language query to search for in the database"
                )
            },
            required=["search_query"]
        )
    )
)]

In [10]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Use the supplied tools to assist the user."
    },
    {
        "role": "user",
        "content": "What did Weaviate release in 1.27?"
    }
]

def generate(lm_client, messages, tools):
    return lm_client.chat.completions.create(
        model="gemini-2.0-flash-exp",
        messages=messages,
        tools=tools
    ).choices[0]


response = generate(client, messages, tools)

print(response)

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"search_query":"What was released in Weaviate version 1.27?"}', name='search_weaviate'), type='function')]))
